In [51]:
# from evaluation_utils.eval_copilot.copilot_utils import create_message, run_chat_completions
# 
# msg = create_message("user","What is chunking in RAG?")
# run_chat_completions([msg])

In [52]:
relevance_system_prompt = """
You are an AI assistant. You will be given the definition of an evaluation metric for assessing the quality of an answer in a question-answering task. Your job is to compute an accurate evaluation score using the provided evaluation metric.
"""

# relevance_user_instructions = """
# Relevance measures how well the answer addresses the main aspects of the question, based on the context. Consider whether all and only the important aspects are contained in the answer when evaluating relevance. Given the context and question, score the relevance of the answer between one to five stars using the following rating scale:
# One star: the answer completely lacks relevance
# Two stars: the answer mostly lacks relevance
# Three stars: the answer is partially relevant
# Four stars: the answer is mostly relevant
# Five stars: the answer has perfect relevance
# 
# This rating value should always be an integer between 1 and 5. So the rating produced should be 1 or 2 or 3 or 4 or 5.
# """

relevance_user_instructions = """
Relevance measures how well the answer addresses the main aspects of the question, based on the context. Consider whether all and only the important aspects are contained in the answer when evaluating relevance. Given the context and question, score the relevance of the answer using the following Likert scale:
- "Very Bad": the answer completely lacks relevance
- "Bad": the answer mostly lacks relevance
- "Average": the answer is partially relevant
- "Good": the answer is mostly relevant
- "Very Good": the answer has perfect relevance

This rating should reflect the degree of relevance. Choose from "Very Bad", "Bad", "Average", "Good", or "Very Good" to represent the relevance score.

"""

relevance_few_shot_examples = [
    {
        "context": "Marie Curie was a Polish-born physicist and chemist who pioneered research on radioactivity and was the first woman to win a Nobel Prize.",
        "question": "What field did Marie Curie excel in?",
        "answer": "Marie Curie was a renowned painter who focused mainly on impressionist styles and techniques.",
        "stars": "Very Bad",
        "justification": "The answer is completely irrelevant as it incorrectly identifies Marie Curie as a painter, whereas she was actually a physicist and chemist. The answer fails to address the main aspect of the question, which is her field of expertise.",
        "improvement_suggestion": "Correct the answer to reflect Marie Curie's true field of expertise. An improved answer would state that Marie Curie was a notable physicist and chemist known for her research on radioactivity."
    },
    {
        "context": "The Beatles were an English rock band formed in Liverpool in 1960, and they are widely regarded as the most influential music band in history.",
        "question": "Where were The Beatles formed?",
        "answer": "The band The Beatles began their journey in London, England, and they changed the history of music.",
        "stars": "Bad",
        "justification": "The answer mostly lacks relevance as it provides incorrect information about the formation location of The Beatles. The question explicitly asks for the place of formation, and the answer incorrectly states London instead of Liverpool.",
        "improvement_suggestion": "Correct the geographical error in the answer. An improved answer would correctly state that The Beatles were formed in Liverpool, England."
    },
    {
        "context": "The recent Mars rover, Perseverance, was launched in 2020 with the main goal of searching for signs of ancient life on Mars. The rover also carries an experiment called MOXIE, which aims to generate oxygen from the Martian atmosphere.",
        "question": "What are the main goals of Perseverance Mars rover mission?",
        "answer": "The Perseverance Mars rover mission focuses on searching for signs of ancient life on Mars.",
        "stars": "Average",
        "justification": "The answer is partially relevant. It correctly identifies one of the main goals of the Perseverance rover – searching for ancient life. However, it omits other key aspects like the MOXIE experiment, which is also a significant part of the mission's goals.",
        "improvement_suggestion": "Expand the answer to include more goals of the mission. An improved answer would mention the MOXIE experiment for generating oxygen from the Martian atmosphere, along with the search for ancient life."
    },
    {
        "context": "The Mediterranean diet is a commonly recommended dietary plan that emphasizes fruits, vegetables, whole grains, legumes, lean proteins, and healthy fats. Studies have shown that it offers numerous health benefits, including a reduced risk of heart disease and improved cognitive health.",
        "question": "What are the main components of the Mediterranean diet?",
        "answer": "The Mediterranean diet primarily consists of fruits, vegetables, whole grains, and legumes.",
        "stars": "Good",
        "justification": "The answer is mostly relevant. It accurately lists several main components of the Mediterranean diet, such as fruits, vegetables, whole grains, and legumes. However, it misses out on mentioning lean proteins and healthy fats, which are also important aspects of this diet.",
        "improvement_suggestion": "Include all key components of the Mediterranean diet in the answer. An improved answer would also mention lean proteins and healthy fats, in addition to the already listed components."
    },
    {
        "context": "The Queen's Royal Castle is a well-known tourist attraction in the United Kingdom. It spans over 500 acres and contains extensive gardens and parks. The castle was built in the 15th century and has been home to generations of royalty.",
        "question": "What are the main attractions of the Queen's Royal Castle?",
        "answer": "The main attractions of the Queen's Royal Castle are its expansive 500-acre grounds, extensive gardens, parks, and the historical castle itself, which dates back to the 15th century and has housed generations of royalty.",
        "stars": "Very Good",
        "justification": "The answer has perfect relevance. It comprehensively addresses all the important aspects of the question, covering the extensive grounds, gardens, parks, the historical significance of the castle, and its connection to generations of royalty, which aligns precisely with the context provided.",
        "improvement_suggestion": "No improvement needed. The answer perfectly addresses the question and aligns with the context provided."
    }
]


relevance_evaluation_prompt = {
    "system": relevance_system_prompt,
    "user": relevance_user_instructions,
    "examples" : relevance_few_shot_examples
}

In [53]:
from evaluation_utils.eval_copilot.copilot_utils import create_message, run_chat_completions

def get_simple_response(user_prompt):
    simple_prompt = create_message("system","You are a helpful assistant.")
    msgs = [simple_prompt, create_message("user", user_prompt)]
    response = run_chat_completions(msgs)
    return response

In [54]:
ground_truth = """
Napoleonic Wars, (1799–1815) Series of wars that ranged France against shifting alliances of European powers. Originally an attempt to maintain French strength established by the French Revolutionary Wars, they became efforts by Napoleon to affirm his supremacy in the balance of European power. A victory over Austria at the Battle of Marengo (1800) left France the dominant power on the continent. Only Britain remained strong, and its victory at the Battle of Trafalgar (1805) ended Napoleon’s threat to invade England. Napoleon won major victories in the Battles of Ulm and Austerlitz (1805), Jena and Auerstedt (1806), and Friedland (1807) against an alliance of Russia, Austria, and Prussia. The resulting Treaties of Tilsit (1807) and the Treaty of Schönbrunn (1809) left most of Europe from the English Channel to the Russian border either part of the French Empire, controlled by France, or allied to it by treaty. Napoleon’s successes resulted from a strategy of moving his army rapidly, attacking quickly, and defeating each of the disconnected enemy units. His enemies’ responding strategy was to avoid engagement while withdrawing, forcing Napoleon’s supply lines to be overextended; the strategy was successfully used against him by the duke of Wellington in the Peninsular War and by Mikhail, Prince Barclay de Tolly, in Russia. In 1813 the Quadruple Alliance formed to oppose Napoleon and amassed armies that outnumbered his. Defeated at the Battle of Leipzig, he was forced to withdraw west of the Rhine River, and after the invasion of France (1814) he abdicated. He rallied a new army to return in the Hundred Days (1815), but a revived Quadruple Alliance opposed him. His final defeat at the Battle of Waterloo was caused by his inability to surprise and to prevent the two armies, led by Wellington and Gebhard von Blücher, from joining forces to defeat him. With his second abdication and exile, the era of the Napoleonic Wars ended.
"""

In [55]:
prompt = "How many countries were involved in the Napoleanic Wars?"
response = get_simple_response(prompt)
print(response)

The Napoleonic Wars involved several countries. They primarily involved France and its allies against a coalition of other European nations. The exact number of countries involved varied throughout the different stages of the conflict. At its height, more than 30 countries were involved in the wars.


In [56]:
from evaluation_utils.eval_copilot.eval_copilot import EvaluationCopilotNew

relevance_judge = EvaluationCopilotNew(relevance_evaluation_prompt)
score, justification = relevance_judge.get_score_and_justification(ground_truth, prompt, response)
print(score, justification)

Full Response: Stars: Very Good
Justification: The answer has perfect relevance. It accurately addresses the question by mentioning that the Napoleonic Wars involved several countries and provides additional context by specifying that it primarily involved France and its allies against a coalition of other European nations. It also mentions that the exact number of countries involved varied throughout the different stages of the conflict, and at its height, more than 30 countries were involved. This aligns precisely with the context provided.
0 The answer has perfect relevance. It accurately addresses the question by mentioning that the Napoleonic Wars involved several countries and provides additional context by specifying that it primarily involved France and its allies against a coalition of other European nations. It also mentions that the exact number of countries involved varied throughout the different stages of the conflict, and at its height, more than 30 countries were involv

In [57]:
msft_relevance_evaluation_prompt = """
System:
You are an AI assistant. You will be given the definition of an evaluation metric for assessing the quality of an answer in a question-answering task. Your job is to compute an accurate evaluation score using the provided evaluation metric.

User:
Relevance measures how well the answer addresses the main aspects of the question, based on the context. Consider whether all and only the important aspects are contained in the answer when evaluating relevance. Given the context and question, score the relevance of the answer between one to five stars using the following rating scale:
One star: the answer completely lacks relevance
Two stars: the answer mostly lacks relevance
Three stars: the answer is partially relevant
Four stars: the answer is mostly relevant
Five stars: the answer has perfect relevance

This rating value should always be an integer between 1 and 5. So the rating produced should be 1 or 2 or 3 or 4 or 5.

context: Marie Curie was a Polish-born physicist and chemist who pioneered research on radioactivity and was the first woman to win a Nobel Prize.
question: What field did Marie Curie excel in?
answer: Marie Curie was a renowned painter who focused mainly on impressionist styles and techniques.
stars: 1

context: The Beatles were an English rock band formed in Liverpool in 1960, and they are widely regarded as the most influential music band in history.
question: Where were The Beatles formed?
answer: The band The Beatles began their journey in London, England, and they changed the history of music.
stars: 2

context: The recent Mars rover, Perseverance, was launched in 2020 with the main goal of searching for signs of ancient life on Mars. The rover also carries an experiment called MOXIE, which aims to generate oxygen from the Martian atmosphere.
question: What are the main goals of Perseverance Mars rover mission?
answer: The Perseverance Mars rover mission focuses on searching for signs of ancient life on Mars.
stars: 3

context: The Mediterranean diet is a commonly recommended dietary plan that emphasizes fruits, vegetables, whole grains, legumes, lean proteins, and healthy fats. Studies have shown that it offers numerous health benefits, including a reduced risk of heart disease and improved cognitive health.
question: What are the main components of the Mediterranean diet?
answer: The Mediterranean diet primarily consists of fruits, vegetables, whole grains, and legumes.
stars: 4

context: The Queen's Royal Castle is a well-known tourist attraction in the United Kingdom. It spans over 500 acres and contains extensive gardens and parks. The castle was built in the 15th century and has been home to generations of royalty.
question: What are the main attractions of the Queen's Royal Castle?
answer: The main attractions of the Queen's Royal Castle are its expansive 500-acre grounds, extensive gardens, parks, and the historical castle itself, which dates back to the 15th century and has housed generations of royalty.
stars: 5

context: {context}
question: {question}
answer: {answer}
stars:
"""

In [58]:
msft_eval_msg = create_message("system", msft_relevance_evaluation_prompt.format(context=ground_truth, question=prompt, answer=response))
msft_score = run_chat_completions([msft_eval_msg])
print(msft_score)

4


In [59]:
msft_eval_msg

{'role': 'system',
 'content': "\nSystem:\nYou are an AI assistant. You will be given the definition of an evaluation metric for assessing the quality of an answer in a question-answering task. Your job is to compute an accurate evaluation score using the provided evaluation metric.\n\nUser:\nRelevance measures how well the answer addresses the main aspects of the question, based on the context. Consider whether all and only the important aspects are contained in the answer when evaluating relevance. Given the context and question, score the relevance of the answer between one to five stars using the following rating scale:\nOne star: the answer completely lacks relevance\nTwo stars: the answer mostly lacks relevance\nThree stars: the answer is partially relevant\nFour stars: the answer is mostly relevant\nFive stars: the answer has perfect relevance\n\nThis rating value should always be an integer between 1 and 5. So the rating produced should be 1 or 2 or 3 or 4 or 5.\n\ncontext: Mar

In [60]:
import re

def parse_score_and_justification_words(full_response):
    score_map = {
        "Very Bad": "1",
        "Bad": "2",
        "Average": "3",
        "Good": "4",
        "Very Good": "5",
    }

    # Adjusting regex to be more flexible with whitespace and line breaks
    score_match = re.search(
        r"Stars:\s*(Very Bad|Bad|Average|Good|Very Good)\s*", full_response, re.DOTALL
    )
    justification_match = re.search(r"Justification:\s*(.+)", full_response, re.DOTALL)

    score = score_map[score_match.group(1).strip()] if score_match else "0"
    justification = (
        justification_match.group(1).strip()
        if justification_match
        else "No justification provided."
    )
    return score, justification

# Testing with your provided full_response
full_response = """
Stars:Very Good
Justification: The answer has perfect relevance. It accurately identifies the major European powers involved in the Napoleonic Wars, including France, Great Britain, Russia, Prussia, Austria, and Spain. Additionally, it provides an estimate of the number of countries involved, stating that around 30 countries were involved in the conflicts. This aligns precisely with the information provided in the context.
"""

score, justification = parse_score_and_justification_words(full_response)
print("Score:", score)
print("Justification:", justification)


Score: 5
Justification: The answer has perfect relevance. It accurately identifies the major European powers involved in the Napoleonic Wars, including France, Great Britain, Russia, Prussia, Austria, and Spain. Additionally, it provides an estimate of the number of countries involved, stating that around 30 countries were involved in the conflicts. This aligns precisely with the information provided in the context.


In [61]:
def parse_response(response_text):
    # Split the response into parts
    parts = response_text.split("\n\n")

    # Extract the explanation
    explanation = parts[0].split("Explanation:\n")[1]

    # Extract the improvement suggestions
    improvement_suggestions = parts[1].split("Improvement Suggestions:\n")[1]

    # Extract the rating
    rating = parts[2].split("Rating: ")[1].strip("[]")

    return rating, explanation, improvement_suggestions

# Example usage
response_text = """Explanation:
The assistant's response provides a clear and accurate overview of the participation in the Napoleonic Wars. It correctly identifies that the number of countries involved can vary depending on the definition of participation. The response also successfully highlights the involvement of major powers such as France, Great Britain, Russia, Austria, Prussia, and Spain, and mentions the participation of other European states and coalitions. This information is relevant and provides a good understanding of the scale and international nature of the Napoleonic Wars. However, the response could have been improved by providing a more specific number or range of countries involved, even if this number is an estimate or varies among historians.

Improvement Suggestions:
The response could be improved by providing a more precise estimate of the number of countries involved, acknowledging that this number can vary among historical sources. Additionally, including a brief mention of some lesser-known countries or states that participated could add depth to the answer.

Rating: [[4]]"""

explanation, improvement_suggestions, rating = parse_response(response_text)
print("Explanation:", explanation)
print("Improvement Suggestions:", improvement_suggestions)
print("Rating:", rating)


Explanation: The assistant's response provides a clear and accurate overview of the participation in the Napoleonic Wars. It correctly identifies that the number of countries involved can vary depending on the definition of participation. The response also successfully highlights the involvement of major powers such as France, Great Britain, Russia, Austria, Prussia, and Spain, and mentions the participation of other European states and coalitions. This information is relevant and provides a good understanding of the scale and international nature of the Napoleonic Wars. However, the response could have been improved by providing a more specific number or range of countries involved, even if this number is an estimate or varies among historians.
Improvement Suggestions: The response could be improved by providing a more precise estimate of the number of countries involved, acknowledging that this number can vary among historical sources. Additionally, including a brief mention of some 